<a href="https://colab.research.google.com/github/Shridipa/-C-Users-Admin-Videos-Captures-project.py---171897560-Codespaces_-potential-space-pancake-/blob/main/MoodMorph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
with open('/content/gdrive/My Drive/KRS/my_file.txt', 'w') as f:
    f.write('This data will persist.')

with open('/content/gdrive/My Drive/KRS/my_file.txt', 'r') as f:
    content = f.read()
print(content)

This data will persist.


In [ ]:
!pip install streamlit scikit-learn pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd

product_df = pd.read_csv('/content/gdrive/MyDrive/KRS/emotionally_intelligent_recommendation_dataset.csv')
product_df.columns = product_df.columns.str.strip().str.lower()

display(product_df.head())


,mbti,mood,gender,age group,product category,example product,product link
0,INTJ,Happy,Male,Teen,Joyful Analytical Tech Essentials for Trendy E...,Joyful Lamp for happy INTJ male teens,https://www.amazon.in/s?k=lamp
1,INTJ,Happy,Male,Adult,Joyful Productivity & Strategy Tools for Aspir...,Joyful Art Kit for happy INTJ male adults,https://www.amazon.in/s?k=art%20kit
2,INTJ,Happy,Male,Senior,Joyful Focus & Growth Gear for Comforting Driv...,Joyful Mug for happy INTJ male seniors,https://www.amazon.in/s?k=mug
3,INTJ,Happy,Female,Teen,Joyful Productivity & Strategy Tools for Trend...,Joyful Headphones for happy INTJ female teens,https://www.amazon.in/s?k=headphones
4,INTJ,Happy,Female,Adult,Joyful Productivity & Strategy Tools for Aspir...,Joyful Journal for happy INTJ female adults,https://www.amazon.in/s?k=journal


In [ ]:
emotion_df = pd.read_csv('/content/gdrive/MyDrive/KRS/mood_match_emotion_dataset.csv')
emotion_map = emotion_df.groupby('Word')['Emotion'].first().to_dict()

def classify_mood(text):
    words = text.lower().split()
    for word in words:
        if word in emotion_map:
            return emotion_map[word].lower()
    return 'unknown'

In [ ]:
def map_to_general_category(product_name):
    name = product_name.lower()
    if "book" in name:
        return "books"
    elif "headphone" in name or "speaker" in name:
        return "tech gadgets"
    elif "skincare" in name or "self-care" in name:
        return "self-care"
    elif "yoga" in name or "fitness" in name or "treadmill" in name:
        return "fitness"
    elif "mug" in name or "personalized" in name:
        return "gifts"
    elif "playstation" in name or "game" in name:
        return "entertainment"
    else:
        return "misc"

product_df['general_category'] = product_df['product category'].apply(map_to_general_category)

In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

X = product_df[['mbti', 'mood', 'gender', 'age group']].copy()
y = product_df['general_category']

encoders = {}
for col in X.columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    encoders[col] = le

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
y_series = pd.Series(y_encoded)
valid_classes = y_series.value_counts()[y_series.value_counts() >= 5].index

X_filtered = X[y_series.isin(valid_classes)]
y_filtered = y_encoded[y_series.isin(valid_classes)]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(
    X_filtered, y_filtered, test_size=0.2, random_state=42, stratify=y_filtered
)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

print("Model Evaluation Metrics")
print(f"- Accuracy: {accuracy:.2f}")
print(f"- Precision: {precision:.2f}")
print(f"- Recall: {recall:.2f}")
print(f"- F1 Score: {f1:.2f}")

Model Evaluation Metrics
- Accuracy: 0.78
- Precision: 0.70
- Recall: 0.78
- F1 Score: 0.73


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

product_df = pd.read_csv('/content/gdrive/MyDrive/KRS/emotionally_intelligent_recommendation_dataset.csv')
emotion_df = pd.read_csv('/content/gdrive/MyDrive/KRS/mood_match_emotion_dataset.csv')
emotion_map = emotion_df.groupby('Word')['Emotion'].first().to_dict()

def classify_mood(text):
    words = text.lower().split()
    for word in words:
        if word in emotion_map:
            return emotion_map[word].lower()
    return 'unknown'

st.set_page_config(page_title="MoodMorph", page_icon="🧠", layout="centered")
st.markdown("<h1 style='text-align: center; color: #4B8BBE;'>🧠 MoodMorph</h1>", unsafe_allow_html=True)
st.markdown("<h4 style='text-align: center; color: #666;'>Your Personalised Product Recommender</h4>", unsafe_allow_html=True)
st.markdown("---")

col1, col2 = st.columns(2)
with col1:
    mbti = st.selectbox("🧬 Select your MBTI type", sorted(product_df['MBTI'].unique()))
    gender = st.selectbox("👤 Select your gender", sorted(product_df['Gender'].unique()))
with col2:
    age_group = st.selectbox("🎂 Select your age group", sorted(product_df['Age Group'].unique()))
    user_text = st.text_input("🗣️ Describe your mood in a sentence")

st.markdown("---")

if st.button("✨ Recommend Product"):
    mood = classify_mood(user_text)
    if mood == 'unknown':
        st.warning("Mood not recognized. Showing fallback recommendation based on personality and demographics.")
    else:
        st.markdown(f"<p style='font-size:18px;'>Detected Mood: <strong>{mood.title()}</strong></p>", unsafe_allow_html=True)

    for col in ['MBTI', 'Mood', 'Gender', 'Age Group', 'Product Category']:
        product_df[col] = product_df[col].astype(str).str.lower()

    input_raw = {
        'MBTI': mbti.lower(),
        'Mood': mood,
        'Gender': gender.lower(),
        'Age Group': age_group.lower()
    }
    input_df = pd.DataFrame([input_raw])

    X = product_df[['MBTI', 'Mood', 'Gender', 'Age Group']].copy()
    y = product_df['Product Category'].astype(str).str.lower()

    encoders = {}
    for col in X.columns:
        le = LabelEncoder()
        le.fit(X[col])
        known_classes = set(le.classes_)
        input_df[col] = input_df[col].apply(lambda x: x if x in known_classes else 'unknown')
        if 'unknown' not in le.classes_:
            le.classes_ = np.append(le.classes_, 'unknown')
        X[col] = le.transform(X[col])
        input_df[col] = le.transform(input_df[col])
        encoders[col] = le

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X, y_encoded)

    predicted_label = model.predict(input_df)[0]
    predicted_category = label_encoder.inverse_transform([predicted_label])[0]
    st.markdown(f"<h4 style='color:#2E8B57;'>🎯 Recommended Product Category: <strong>{predicted_category.title()}</strong></h4>", unsafe_allow_html=True)

    match = product_df[
        (product_df['MBTI'] == input_raw['MBTI']) &
        (product_df['Mood'] == input_raw['Mood']) &
        (product_df['Gender'] == input_raw['Gender']) &
        (product_df['Age Group'] == input_raw['Age Group']) &
        (product_df['Product Category'] == predicted_category)
    ]

    if not match.empty:
        st.markdown(f"<p style='font-size:16px;'>🛍️ <strong>Example Product:</strong> {match.iloc[0]['Example Product']}</p>", unsafe_allow_html=True)
        link = match.iloc[0]['Product Link']
        if pd.notnull(link) and str(link).startswith("http"):
            st.markdown(f"<a href='{link}' target='_blank'><button style='background-color:#4B8BBE;color:white;padding:10px;border:none;border-radius:5px;'>🛒 Buy Now</button></a>", unsafe_allow_html=True)
        else:
            st.warning("Product link is missing or invalid.")
    else:
        fallback = product_df[product_df['Product Category'] == predicted_category]
        if not fallback.empty:
            st.markdown(f"<p style='font-size:16px;'>🛍️ <strong>Example Product:</strong> {fallback.iloc[0]['Example Product']}</p>", unsafe_allow_html=True)
            link = fallback.iloc[0]['Product Link']
            if pd.notnull(link) and str(link).startswith("http"):
                st.markdown(f"<a href='{link}' target='_blank'><button style='background-color:#4B8BBE;color:white;padding:10px;border:none;border-radius:5px;'>🛒 Buy Now</button></a>", unsafe_allow_html=True)
            else:
                st.warning("Fallback product link is missing or invalid.")
        else:
            st.warning("No product found in the predicted category.")

st.markdown("---")
st.markdown("<p style='text-align:center;font-size:13px;color:#999;'>Powered by MoodMorph • AI meets personality</p>", unsafe_allow_html=True)

Writing app.py


In [ ]:
!streamlit run app.py &>/dev/null &


In [ ]:
!ngrok config add-authtoken 2yD86jsPkGLe3vzqTagGyEFaJbA_4tvu4vKjB9Aqrjcmddd7N

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok
import time

time.sleep(5)
public_url = ngrok.connect(8501)
print("Your app is live at:", public_url)



Your app is live at: NgrokTunnel: "https://9bf8ecf96107.ngrok-free.app" -> "http://localhost:8501"
